In [5]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [6]:
#/content/CommentToxicity-main.zip

In [7]:
df=pd.read_csv('train.csv')

In [8]:
df.iloc[0]['comment_text']

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [9]:
df[df.columns[2:]].iloc[0]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 0, dtype: int64

In [10]:
df[df['toxic']==1].head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [11]:
df.iloc[6].comment_text

'COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK'

In [12]:
from tensorflow.keras.layers import TextVectorization

In [13]:
df.columns[2:].values

array(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'], dtype=object)

In [14]:
df[df.columns[2:]]

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [15]:
df[df.columns[2:]].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [16]:
X=df['comment_text']
y=df[df.columns[2:]].values

In [17]:
X

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159566    ":::::And for the second time of asking, when ...
159567    You should be ashamed of yourself \n\nThat is ...
159568    Spitzer \n\nUmm, theres no actual article for ...
159569    And it looks like it was actually you who put ...
159570    "\nAnd ... I really don't think you understand...
Name: comment_text, Length: 159571, dtype: object

In [18]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [19]:
MAX_FEATURES = 100000

In [20]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,output_sequence_length=1800,output_mode='int')

In [21]:
X.values

array(["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",
       "D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)",
       "Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",
       ...,
       'Spitzer \n\nUmm, theres no actual article for prostitution ring.  - Crunch Captain.',
       'And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.',
       '"\nAnd ... I really don\'t think you understand.  I came here and my idea was bad right away.  What kind of communit

In [22]:
vectorizer.adapt(X.values)

In [23]:
vectorizer('Real Madrid, is champion of Europe')[:6]

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([ 337, 7427,    9, 3857,    4, 1310])>

In [24]:
vectorized_text = vectorizer(X.values)

In [25]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [26]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)#each batch has 16 samples
dataset = dataset.prefetch(8)


In [27]:
dataset.as_numpy_iterator().next()

(array([[   2,  582,   31, ...,    0,    0,    0],
        [4320,    8,  598, ...,    0,    0,    0],
        [   8,   19,  190, ...,    0,    0,    0],
        ...,
        [ 171,    8,   51, ...,    0,    0,    0],
        [  82,   20,    7, ...,    0,    0,    0],
        [   8,   47,  739, ...,    0,    0,    0]]), array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

In [28]:
batch_X,batch_y = dataset.as_numpy_iterator().next()

In [29]:
batch_X.shape

(16, 1800)

In [30]:
batch_y.shape

(16, 6)

In [31]:
len(dataset)*16 #amount of data= batches * 16 samples each

159584

In [32]:
len(dataset) #=batches

9974

In [33]:
int(len(dataset)*.7)#batches

6981

In [34]:
int(len(dataset)*.2)#batches

1994

In [35]:
int(len(dataset)*.1)#batches

997

In [36]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [37]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [38]:
model = Sequential()
model.add(Embedding(MAX_FEATURES+1,32))
model.add(Bidirectional(LSTM(32,activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))


In [39]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          3200032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [41]:
history = model.fit(train,epochs=1,validation_data=val)

6981/6981 [==============================] - 620s 88ms/step - loss: 0.0622 - val_loss: 0.0472


In [42]:
from matplotlib import pyplot as plt

In [43]:
history.history

{'loss': [0.06222538650035858], 'val_loss': [0.04721541330218315]}

In [47]:
#plt.figure(figsize=(8,5))
#pd.DataFrame(history.history).plot()
#plt.show()

In [48]:
input_text = vectorizer('You freaking suck! I am going to hit you.')

In [49]:
input_text

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([   7, 7158,  397, ...,    0,    0,    0])>

In [52]:
np.array(input_text)

array([   7, 7158,  397, ...,    0,    0,    0])

In [50]:
[input_text]

[<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([   7, 7158,  397, ...,    0,    0,    0])>]

In [51]:
np.array(input_text)

array([   7, 7158,  397, ...,    0,    0,    0])

In [54]:
results = model.predict(np.array([input_text]))

1/1 [==============================] - 1s 682ms/step


In [55]:
results

array([[0.99070895, 0.12268436, 0.89398384, 0.03094537, 0.7207574 ,
        0.08931608]], dtype=float32)

In [59]:
np.expand_dims(input_text,0)

array([[   7, 7158,  397, ...,    0,    0,    0]])

In [60]:
results = model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 61ms/step


In [61]:
results

array([[0.99070895, 0.12268436, 0.89398384, 0.03094537, 0.7207574 ,
        0.08931608]], dtype=float32)

In [62]:
batch = test.as_numpy_iterator().next()
batch

(array([[  47,    7,  466, ...,    0,    0,    0],
        [  41,   47,  119, ...,    0,    0,    0],
        [   8,   55,   72, ...,    0,    0,    0],
        ...,
        [ 702,   46,   14, ...,    0,    0,    0],
        [5382,   35,   14, ...,    0,    0,    0],
        [1211,   10,   12, ...,    0,    0,    0]]), array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]))

In [63]:
batch_X,batch_y=test.as_numpy_iterator().next()

In [64]:
model.predict(batch_X)

1/1 [==============================] - 0s 64ms/step


array([[6.17103884e-03, 5.38921495e-06, 2.49126111e-04, 2.46896379e-05,
        8.14493178e-05, 9.82756173e-05],
       [1.59400329e-01, 7.71641324e-04, 1.29379313e-02, 1.92399044e-03,
        1.12060625e-02, 4.89357952e-03],
       [6.21885099e-02, 3.91168986e-04, 5.82353352e-03, 1.08193536e-03,
        3.57252359e-03, 2.70885043e-03],
       [7.07961246e-03, 5.64701259e-06, 2.61615758e-04, 2.59475310e-05,
        9.28213194e-05, 1.03806793e-04],
       [4.60680760e-03, 2.59286526e-06, 1.49237399e-04, 1.31565102e-05,
        4.72889478e-05, 5.70169868e-05],
       [1.49378208e-02, 3.57242207e-05, 9.22743522e-04, 1.25734703e-04,
        3.68077133e-04, 4.09576664e-04],
       [7.04508414e-03, 7.29212934e-06, 2.99036474e-04, 3.10682408e-05,
        1.00104946e-04, 1.20559947e-04],
       [1.95623096e-03, 5.95457209e-07, 5.20424619e-05, 3.57116915e-06,
        1.25249426e-05, 1.79247727e-05],
       [7.64169171e-02, 1.60497293e-04, 4.10419703e-03, 4.89398139e-04,
        2.96533830e-03, 

In [65]:
(model.predict(batch_X)>0.5).astype(int)

1/1 [==============================] - 0s 65ms/step


array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [66]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 1, 1]])

In [67]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [68]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [69]:
for batch in test.as_numpy_iterator():
  X_true,y_true=batch
  yhat=model.predict(X_true)

  y_true = y_true.flatten()
  yhat=yhat.flatten()

  pre.update_state(y_true, yhat)
  re.update_state(y_true, yhat)
  acc.update_state(y_true, yhat)


1/1 [==============================] - 0s 60ms/step


In [70]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

Precision: 0.830118715763092, Recall: 0.6502033472061157, Accuracy: 0.47342026233673096
